In [1]:
!pip install -U statsmodels

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [3]:
data = pd.read_csv('/content/drive/MyDrive/PROJECT/Dataset/train.csv', parse_dates=['Order Date'], dayfirst=True)

In [4]:
data.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,2017-11-08,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,2017-11-08,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,2017-06-12,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,2016-10-11,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,2016-10-11,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [5]:
# Convert 'Order Date' to datetime
data["Order Date"] = pd.to_datetime(data["Order Date"], errors='coerce')

# Ensure the frequency is set correctly
data.set_index("Order Date", inplace=True)

# Aggregate sales data by year
data['Year'] = data.index.year

In [9]:
sales_by_category_year = data.groupby(["Category", "Year"])["Sales"].sum().reset_index()

# Initialize forecast dataframe
forecasted_sales = pd.DataFrame(columns=["Category", "Forecasted_sales_2019"])
result = []

for category in sales_by_category_year["Category"].unique():
    category_data = sales_by_category_year[sales_by_category_year["Category"] == category]
    sales_series = category_data.set_index("Year")["Sales"]

    train_data, valid_data = train_test_split(sales_series, test_size=0.2, random_state=42, shuffle=False)

    # Fit the Exponential Smoothing model
    model = ExponentialSmoothing(train_data,
                                 initialization_method='legacy-heuristic',
                                 trend='add')
    res_model = model.fit()
    forecast = res_model.forecast(1)  # Forecasting for the next year (2019)

    forecast_df = pd.DataFrame({
        "Category": [category],
        "Forecasted_sales_2019": [forecast.values[0]]
    })

    forecasted_sales = pd.concat([forecasted_sales, forecast_df], ignore_index=True)

    true_values = valid_data
    # Use start and end parameters instead of just the length
    predicted_values = res_model.predict(start=valid_data.index[0], end=valid_data.index[-1])
    mae = mean_absolute_error(true_values, predicted_values)
    residuals = true_values - predicted_values
    result.append({
        "Category": category,
        'True Values': true_values.values,
        'Predicted Values': predicted_values.values,
        "MAE": mae,
        "Residuals": residuals.values  #Positive value means model underpredicted, Negative value means model overpredicted
    })
    for i in result:
        print(f"Category: {i['Category']}")
        print(f"True Values: {i['True Values']}")
        print(f"Predicted Values: {i['Predicted Values']}")
        print(f"MAE: {i['MAE']}")
        print(f"Residuals: {i['Residuals']}")
        print()


# Format the forecasted sales
forecasted_sales["Forecasted_sales_2019"] = forecasted_sales["Forecasted_sales_2019"].apply(lambda x: '{:,.2f}'.format(x))
print("Forecasted Sales in 2019 for Each Product Category")
print(forecasted_sales)

Category: Furniture
True Values: [212313.7872]
Predicted Values: [15452403.63416194]
MAE: 15240089.84696194
Residuals: [-15240089.84696194]

Category: Furniture
True Values: [212313.7872]
Predicted Values: [15452403.63416194]
MAE: 15240089.84696194
Residuals: [-15240089.84696194]

Category: Office Supplies
True Values: [240367.541]
Predicted Values: [43422575.27609382]
MAE: 43182207.73509382
Residuals: [-43182207.73509382]

Category: Furniture
True Values: [212313.7872]
Predicted Values: [15452403.63416194]
MAE: 15240089.84696194
Residuals: [-15240089.84696194]

Category: Office Supplies
True Values: [240367.541]
Predicted Values: [43422575.27609382]
MAE: 43182207.73509382
Residuals: [-43182207.73509382]

Category: Technology
True Values: [269370.691]
Predicted Values: [57893489.87108108]
MAE: 57624119.180081084
Residuals: [-57624119.18008108]

Forecasted Sales in 2019 for Each Product Category
          Category Forecasted_sales_2019
0        Furniture            186,789.66
1  Office 

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Pr